In [ ]:
# notebooks/01_data_ingestion_and_cleaning.ipynb

# Cell 1: Setup and Imports
import sys
import os
import logging
import pandas as pd 

# Add the 'src' directory to the Python path
current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from src.data_loader import load_data_from_csv, load_data_from_postgres, download_kaggle_dataset
from src.data_cleaner import clean_telco_data
from src.utils import logger, get_output_base_path

logger.setLevel(logging.DEBUG)

In [ ]:
# Cell 2: Initialize Output Paths for this Run
output_base_path_for_run = get_output_base_path(project_root)
logger.info(f"All pipeline outputs for this run will be saved under: {output_base_path_for_run}")


In [ ]:
# Cell 3: Download Raw Data from Kaggle (THIS IS THE CELL YOU NEED TO ADD/ENSURE IS PRESENT)
kaggle_dataset_name = "blastchar/telco-customer-churn"
data_folder_path = os.path.join(project_root, 'data') # This is where the CSV will be saved

# This function will attempt to download the dataset if it's not already there
# and return the path to the downloaded CSV file.
downloaded_csv_path = download_kaggle_dataset(kaggle_dataset_name, data_folder_path)

if downloaded_csv_path:
    logger.info(f"Kaggle dataset available/downloaded at: {downloaded_csv_path}")
    # Now that we know the path, we can use it in the next cell
    csv_file_path = downloaded_csv_path
else:
    logger.error("Failed to ensure Kaggle dataset is available. CSV loading might fail.")
    csv_file_path = os.path.join(data_folder_path, 'WA_Fn-UseC_-Telco-Customer-Churn.csv') # Fallback path

In [ ]:
# Cell 4: Data Ingestion (CSV)
csv_file_path = os.path.join(project_root, 'data', 'WA_Fn-UseC_-Telco-Customer-Churn.csv')
telco_df_raw = load_data_from_csv(csv_file_path, output_base_path=output_base_path_for_run)

if telco_df_raw is not None:
    logger.info(f"Raw DataFrame loaded successfully with shape: {telco_df_raw.shape}")
else:
    logger.error("Failed to load raw data from CSV. Check previous logs.")


In [ ]:
# Cell 5: Data Cleaning and Transformation
if telco_df_raw is not None:
    telco_df_cleaned = clean_telco_data(telco_df_raw, output_base_path=output_base_path_for_run)

    if telco_df_cleaned is not None:
        logger.info(f"Cleaned DataFrame created successfully with shape: {telco_df_cleaned.shape}")
        logger.info(f"Final check: Number of missing values after cleaning:\n{telco_df_cleaned.isnull().sum().to_string()}")
        logger.info(f"Final check: 'TotalCharges' dtype: {telco_df_cleaned['TotalCharges'].dtype}")
        logger.info(f"Final check: 'Churn' unique values: {telco_df_cleaned['Churn'].unique()}")
    else:
        logger.error("Failed to clean data. Check previous logs.")
else:
    logger.warning("Skipping data cleaning as raw data loading failed.")





In [ ]:
# Cell 6: Data Ingestion (PostgreSQL Example - Uses SQL file)
# NOTE: This cell requires your PostgreSQL database (telco_insights_db) to be set up
#       with the 'customer_churn' table and data loaded into it.
#       Only uncomment and run this cell if you are testing the PostgreSQL pathway.

if telco_df_raw is None: # Only run if CSV loading failed or skipped
    logger.info("Attempting to load data from PostgreSQL using a query from file .")

    sql_file_path = os.path.join(project_root, 'sql', 'initial_churn_data.sql')

    sql_query_from_file = None
    try:
        with open(sql_file_path, 'r') as file:
            sql_query_from_file = file.read()
        logger.info(f"Successfully loaded SQL query from: {sql_file_path}")
    except FileNotFoundError:
        logger.error(f"Error: SQL file '{sql_file_path}' not found. Please create it.", exc_info=True)
    except Exception as e:
        logger.error(f"An unexpected error occurred while reading SQL file: {e}", exc_info=True)

    if sql_query_from_file:
        telco_df_sql_raw = load_data_from_postgres(query=sql_query_from_file,
                                                    output_base_path=output_base_path_for_run)

        if telco_df_sql_raw is not None:
            logger.info(f"Raw DataFrame loaded from PostgreSQL successfully with shape: {telco_df_sql_raw.shape}")
        else:
            logger.error("Failed to load raw data from PostgreSQL. Check previous logs.")
    else:
        logger.error("Skipping PostgreSQL data load as SQL query could not be loaded.")

In [ ]:
# Cell 7: Save Cleaned Data
# Assuming output_base_path_for_run and logger are already defined in previous cells

if telco_df_cleaned is not None:
    # Define the output path for the cleaned data
    cleaned_data_output_dir = os.path.join(output_base_path_for_run, 'processed_data')
    os.makedirs(cleaned_data_output_dir, exist_ok=True) # Create directory if it doesn't exist

    cleaned_data_file_path = os.path.join(cleaned_data_output_dir, 'telco_customer_churn_cleaned.parquet')

    try:
        telco_df_cleaned.to_parquet(cleaned_data_file_path, index=False)
        logger.info(f"Cleaned data successfully saved to: {cleaned_data_file_path}")
    except Exception as e:
        logger.error(f"Failed to save cleaned data to Parquet: {e}", exc_info=True)

    # You could also save as CSV if preferred, for broader compatibility:
    # cleaned_data_csv_file_path = os.path.join(cleaned_data_output_dir, 'telco_customer_churn_cleaned.csv')
    # try:
    #     telco_df_cleaned.to_csv(cleaned_data_csv_file_path, index=False)
    #     logger.info(f"Cleaned data also saved to CSV: {cleaned_data_csv_file_path}")
    # except Exception as e:
    #     logger.error(f"Failed to save cleaned data to CSV: {e}", exc_info=True)

else:
    logger.warning("Skipping saving cleaned data as no cleaned DataFrame was found.")